In [ ]:
from xgboost import XGBClassifier

In [ ]:
hidden_states_by_layer = {}
NUM_PROMPTS = 10

for prompt_id in range(1, NUM_PROMPTS + 1):
    with np.load(f'../hidden_states/prompt_{prompt_id}.npz') as loaded_data:
        for i in tqdm(range(1000)):
            curr_hidden_states = loaded_data[f"arr_{i}"][0]
#             print(curr_hidden_states.shape)
            
            #By layer
            for layer in range(1):
                padded_arr = np.zeros((max_story_len, 512))
                padded_arr_len = len(curr_hidden_states[layer][0])
                
                padded_arr[:padded_arr_len] = curr_hidden_states[layer][0]
                
                padded_arr = padded_arr.astype('float32') #FAISS expects data in type float32 instead of float64 - saves memory too!
                
                if(f"layer_{layer}" in hidden_states_by_layer):
                    hidden_states_by_layer[f"layer_{layer}"].append(padded_arr)
                else:
                    hidden_states_by_layer[f"layer_{layer}"] = [padded_arr]

In [ ]:
layer_hs_array = np.array(hidden_states_by_layer["layer_0"])
layer_hs_array.shape

In [ ]:
#Context-level analysis
for context_level in range(len(layer_hs_array)):
    layer_hs_upto_context = layer_hs_array[:context_level]
    

In [ ]:
classifier = self.MODEL_CLASSIFIER(**ht_best_params, seed = 42, objective = 'multi:softmax', eval_metric = "merror")
classifier.fit(self.X_train, self.train_y)
preds = classifier.predict(self.X_test)